In [1]:
import eos
import numpy as np

import cv2

import os.path as osp

import matplotlib.pyplot as plt


In [2]:
def read_pts(filename):
    """A helper function to read the 68 ibug landmarks from a .pts file."""
    lines = open(filename).read().splitlines()
    lines = lines[3:71]

    landmarks = []
    ibug_index = 1  # count from 1 to 68 for all ibug landmarks
    for l in lines:
        coords = l.split()
        landmarks.append(eos.core.Landmark(str(ibug_index), [float(coords[0]), float(coords[1])]))
        ibug_index = ibug_index + 1

    return landmarks

In [3]:
landmarks = read_pts('../bin/data/image_0010.pts')
image_width = 1280 # Make sure to adjust these when using your own images!
image_height = 1024
output_basename = 'out'


In [4]:

model = eos.morphablemodel.load_model("../share/sfm_shape_3448.bin")

In [ ]:
type(model)

In [ ]:
dir(model)

In [7]:
blendshapes = eos.morphablemodel.load_blendshapes("../share/expression_blendshapes_3448.bin")

In [ ]:
type(blendshapes)

In [ ]:
len(blendshapes)

In [ ]:
blendshapes

In [ ]:
type(blendshapes[0])

In [ ]:
dir(blendshapes[0])

In [ ]:
blendshapes[0].name

In [ ]:
blendshapes[0].deformation

In [ ]:
type(blendshapes[0].deformation)

In [ ]:
blendshapes[0].deformation.shape

In [ ]:
for bs in blendshapes:
    print(f'--> name: {bs.name}')
    print(f'    deformation.shape: {bs.deformation.shape}')

In [18]:
# Create a MorphableModel with expressions from the loaded neutral model and blendshapes:
morphablemodel_with_expressions = eos.morphablemodel.MorphableModel(model.get_shape_model(), blendshapes,
                                                                    color_model=eos.morphablemodel.PcaModel(),
                                                                    vertex_definitions=None,
                                                                    texture_coordinates=model.get_texture_coordinates())

In [ ]:
type(morphablemodel_with_expressions)

In [ ]:
dir(morphablemodel_with_expressions)

In [21]:
landmark_mapper = eos.core.LandmarkMapper('../share/ibug_to_sfm.txt')

In [ ]:
type(landmark_mapper)

In [ ]:
dir(landmark_mapper)

In [ ]:
landmark_mapper.get_mappings()

In [25]:
edge_topology = eos.morphablemodel.load_edge_topology('../share/sfm_3448_edge_topology.json')

In [ ]:
type(edge_topology)

In [ ]:
dir(edge_topology)

In [ ]:
type(edge_topology.adjacent_faces)

In [ ]:
np.array(edge_topology.adjacent_faces).shape

In [ ]:
type(edge_topology.adjacent_vertices)

In [ ]:
np.array(edge_topology.adjacent_vertices).shape

In [32]:
contour_landmarks = eos.fitting.ContourLandmarks.load('../share/ibug_to_sfm.txt')


In [ ]:
type(contour_landmarks)

In [ ]:
dir(contour_landmarks)

In [35]:
# contour_landmarks.left_contour

In [36]:
model_contour = eos.fitting.ModelContour.load('../share/sfm_model_contours.json')


In [ ]:
type(model_contour)

In [ ]:
dir(model_contour)

In [39]:

(mesh, pose, shape_coeffs, blendshape_coeffs) = eos.fitting.fit_shape_and_pose(morphablemodel_with_expressions,
    landmarks, landmark_mapper, image_width, image_height, edge_topology, contour_landmarks, model_contour)


In [ ]:
type(mesh)

In [ ]:
dir(mesh)

In [ ]:
attrs = [ 
    'colors',
    'tci',
    'texcoords',
    'tti',
    'tvi',
    'vertices'
 ]

for attr in attrs:
    print('--> ', attr, type(getattr(mesh, attr)))
    print(' ' * 4, attr, np.array(getattr(mesh, attr)).shape)

In [ ]:
type(pose)

In [ ]:
dir(pose)

In [ ]:
type(shape_coeffs)

In [ ]:
len(shape_coeffs)

In [ ]:
shape_coeffs

In [ ]:
type(blendshape_coeffs)

In [ ]:
len(blendshape_coeffs)

In [ ]:
blendshape_coeffs

In [ ]:
output_path = output_basename + '.obj'
print(f'--> Writing output to {output_path}')
eos.core.write_obj(mesh, output_path)
print(f'--> Wrote output to {output_path}')

In [52]:

# Now you can use your favourite plotting/rendering library to display the fitted mesh, using the rendering
# parameters in the 'pose' variable.

# Or for example extract the texture map, like this:
image = cv2.imread('../bin/data/image_0010.png')
out_image = image.copy() # for drawing wireframe


In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB, 3))

In [54]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA, 4)  # extract_texture(...) expects a 4-channel image
texturemap = eos.render.extract_texture(mesh, pose, image)


In [ ]:
type(texturemap)

In [ ]:
texturemap.shape

In [ ]:
eos.render.draw_wireframe?

In [ ]:
eos.fitting.get_3x4_affine_camera_matrix?

In [59]:
# Draw the fitted mesh as wireframe, and save the image:
out_image = eos.render.draw_wireframe(
    out_image, 
    mesh, 
    pose.get_modelview(), 
    pose.get_projection(), 
    np.array((0, image_height, image_width, -image_height))
)

In [ ]:

plt.imshow(cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB, 3))

In [ ]:
output_path = output_basename + '_wireframe.png'
cv2.imwrite(output_path, out_image)

In [ ]:

# Save the mesh as textured obj:
output_path = output_basename + '_with_texture.obj'
print(f'--> Writing output to {output_path}')
eos.core.write_textured_obj(mesh, output_path)
print(f'--> Wrote output to {output_path}')


In [ ]:
# And save the texture as a png:
output_path = osp.splitext(output_path)[0] + '.texture.png'
cv2.imwrite(output_path, texturemap)